In [1]:
import numpy as np
np.random.seed(1337)
from keras import backend as K
from keras.optimizers import SGD
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Activation, Dropout, Flatten, Dense

Using TensorFlow backend.
/Users/vkhanna/Documents/GitHub/Keras_SmileFace_Challenge/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/vkhanna/Documents/GitHub/Keras_SmileFace_Challenge/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/vkhanna/Documents/GitHub/Keras_SmileFace_Challenge/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) /

In [2]:
# Could use larger dimensions, but will make training
# times much much longer
img_width, img_height = (75, 75)

In [3]:
train_dir = 'FEI/train'
test_dir = 'FEI/test'

epochs = 40
batch_size = 8
num_train_samples = 320
num_test_samples  =  80

# perform random transformations so that the
# data is more varied
train_datagen = ImageDataGenerator(rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

test_datagen  = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size,
                                                    class_mode='binary',
                                                    shuffle=True)

validation_generator = test_datagen.flow_from_directory(test_dir,
                                                        target_size=(img_width, img_height),
                                                        batch_size=batch_size,
                                                        class_mode='binary',
                                                        shuffle=True)

Found 320 images belonging to 2 classes.
Found 80 images belonging to 2 classes.


In [4]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [5]:
input_shape

(75, 75, 3)

In [6]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), activation="relu", input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (5, 5), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Convolution is done, so make the fully connected layer
model.add(Flatten())

# Drop 50% of the neurons
model.add(Dense(25, activation="relu"))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# compilation
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

W0727 23:11:17.396481 140737063089088 deprecation_wrapper.py:119] From /Users/vkhanna/Documents/GitHub/Keras_SmileFace_Challenge/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0727 23:11:17.418045 140737063089088 deprecation_wrapper.py:119] From /Users/vkhanna/Documents/GitHub/Keras_SmileFace_Challenge/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0727 23:11:17.422518 140737063089088 deprecation_wrapper.py:119] From /Users/vkhanna/Documents/GitHub/Keras_SmileFace_Challenge/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0727 23:11:17.452540 140737063089088 deprecation_wrapper.py:119] From /Users/vkhanna/Documents/GitHub/Keras_SmileFace_Challenge

In [7]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
checkpoint = ModelCheckpoint("best_model.hdf5", monitor='loss', verbose=1, save_best_only=True, mode='auto',period=1)

model.fit_generator(train_generator,
                    steps_per_epoch=num_train_samples // batch_size,
                    epochs=epochs,
                    validation_data=validation_generator,
                    validation_steps=num_test_samples // batch_size,
                    callbacks=[early_stopping, checkpoint])

W0727 23:11:18.244196 140737063089088 deprecation_wrapper.py:119] From /Users/vkhanna/Documents/GitHub/Keras_SmileFace_Challenge/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/40
40/40 [==============================] - 2s 60ms/step - loss: 0.6915 - acc: 0.5594 - val_loss: 0.6998 - val_acc: 0.5000
Epoch 2/40
40/40 [==============================] - 3s 76ms/step - loss: 0.6846 - acc: 0.5375 - val_loss: 0.6523 - val_acc: 0.5375
Epoch 3/40
40/40 [==============================] - 2s 61ms/step - loss: 0.6341 - acc: 0.6562 - val_loss: 0.5832 - val_acc: 0.6625
Epoch 4/40
40/40 [==============================] - 2s 59ms/step - loss: 0.6677 - acc: 0.5844 - val_loss: 0.5303 - val_acc: 0.8000
Epoch 5/40
40/40 [==============================] - 2s 58ms/step - loss: 0.6490 - acc: 0.6625 - val_loss: 0.5848 - val_acc: 0.7375
Epoch 6/40
40/40 [==============================] - 3s 68ms/step - loss: 0.6126 - acc: 0.6875 - val_loss: 0.4622 - val_acc: 0.8125
Epoch 7/40
40/40 [==============================] - 3s 80ms/step - loss: 0.6223 - acc: 0.6438 - val_loss: 0.4227 - val_acc: 0.7750
Epoch 8/40
40/40 [==============================] - 3s 68ms/step - loss: 0.6047 - a

In [8]:
model.save("model.h5")

In [9]:
model = load_model('model.h5')

In [17]:
test_loss, metrics = model.evaluate_generator(validation_generator, steps=num_test_samples)

In [18]:
print("Test loss {}".format(test_loss))
print("Metrics {}: {} ".format(model.metrics, metrics))

Test loss 0.3412948092445731
Metrics ['accuracy']: 0.8609375 
